<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Inference, Partial Fit, and Fine-Tuning with tsai-rs

This notebook demonstrates how to:
1. **Save and load models** for later inference
2. **Continue training** (partial fit) on new data
3. **Fine-tune** pretrained models on new datasets

## Purpose

In real-world applications, you often need to:
- Deploy models and generate predictions (inference)
- Update models with new data (incremental learning)
- Adapt pretrained models to new tasks (transfer learning)

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
from pathlib import Path

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Load Data

In [ ]:
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

n_vars = X_train.shape[1]
seq_len = X_train.shape[2]
n_classes = len(np.unique(y_train))

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Classes: {n_classes}")

In [ ]:
# Standardize
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)

## Part 1: Model Training and Saving

In [ ]:
# Create model configuration
model_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)

# Create training configuration
learner_config = tsai_rs.LearnerConfig(
    lr=1e-3,
    weight_decay=0.01,
    grad_clip=1.0
)

# Create datasets
train_ds = tsai_rs.TSDataset(X_train_std, y_train)
test_ds = tsai_rs.TSDataset(X_test_std, y_test)

print(f"Model config: {model_config}")
print(f"Learner config: {learner_config}")

### Simulate Training

In [ ]:
# In a real scenario, you would train the model here
# For demonstration, we'll simulate training metrics

def simulate_training_history(n_epochs=10):
    """Simulate training history."""
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_accuracy': []
    }
    
    for epoch in range(n_epochs):
        train_loss = 1.0 * np.exp(-epoch/3) + np.random.normal(0, 0.02)
        val_loss = 1.1 * np.exp(-epoch/3) + np.random.normal(0, 0.03)
        val_acc = 0.5 + 0.4 * (1 - np.exp(-epoch/3)) + np.random.normal(0, 0.01)
        
        history['train_loss'].append(max(0, train_loss))
        history['val_loss'].append(max(0, val_loss))
        history['val_accuracy'].append(np.clip(val_acc, 0, 1))
        
        print(f"Epoch {epoch+1:2d}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")
    
    return history

print("Training model...")
print("-" * 60)
history = simulate_training_history(10)

### Save Model Configuration

In [ ]:
import json

# Create models directory
models_path = Path('models')
models_path.mkdir(exist_ok=True)

# Save model configuration
model_info = {
    'architecture': 'InceptionTimePlus',
    'n_vars': n_vars,
    'seq_len': seq_len,
    'n_classes': n_classes,
    'dataset': dsid,
    'training': {
        'lr': 1e-3,
        'weight_decay': 0.01,
        'n_epochs': 10,
        'final_accuracy': history['val_accuracy'][-1]
    }
}

with open(models_path / 'model_config.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("Model configuration saved to models/model_config.json")
print(json.dumps(model_info, indent=2))

## Part 2: Inference (Generating Predictions)

In [ ]:
# Load model configuration
with open(models_path / 'model_config.json', 'r') as f:
    loaded_config = json.load(f)

print("Loaded model configuration:")
print(json.dumps(loaded_config, indent=2))

In [ ]:
# Recreate model configuration from saved info
inference_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=loaded_config['n_vars'],
    seq_len=loaded_config['seq_len'],
    n_classes=loaded_config['n_classes']
)

print(f"Inference config: {inference_config}")

### Generate Predictions

In [ ]:
def simulate_predictions(X, n_classes):
    """Simulate model predictions."""
    n_samples = len(X)
    
    # Generate random probabilities
    probs = np.random.rand(n_samples, n_classes)
    probs = probs / probs.sum(axis=1, keepdims=True)  # Normalize
    
    # Get predicted classes
    preds = np.argmax(probs, axis=1)
    
    return probs, preds

# Generate predictions on test data
probs, preds = simulate_predictions(X_test_std, n_classes)

print(f"Predictions shape: {preds.shape}")
print(f"Probabilities shape: {probs.shape}")
print(f"\nFirst 10 predictions: {preds[:10]}")
print(f"First 10 true labels: {y_test[:10]}")

In [ ]:
# Single sample inference
single_sample = X_test_std[0:1]  # Shape: (1, n_vars, seq_len)
single_probs, single_pred = simulate_predictions(single_sample, n_classes)

print(f"Single sample shape: {single_sample.shape}")
print(f"Prediction: {single_pred[0]}")
print(f"Probabilities: {single_probs[0]}")

## Part 3: Partial Fit (Incremental Learning)

Continue training with new data while preserving learned knowledge.

In [ ]:
# Simulate new incoming data
# In practice, this would be newly collected samples
n_new_samples = 50
new_X = X_train_std[:n_new_samples].copy()  # Simulate new data
new_y = y_train[:n_new_samples].copy()

# Add some augmentation to simulate different data
new_X = tsai_rs.add_gaussian_noise(new_X, std=0.1, seed=42)

print(f"New data shape: {new_X.shape}")
print(f"New labels: {np.unique(new_y)}")

In [ ]:
# Configuration for partial fit (typically lower learning rate)
partial_fit_config = tsai_rs.LearnerConfig(
    lr=1e-4,  # Lower LR for fine-tuning
    weight_decay=0.01,
    grad_clip=1.0
)

print(f"Partial fit config: {partial_fit_config}")
print("\nKey considerations for partial fit:")
print("  1. Use lower learning rate (e.g., 1e-4 vs 1e-3)")
print("  2. Train for fewer epochs")
print("  3. Monitor for catastrophic forgetting")

In [ ]:
# Simulate partial fit training
print("\nPartial fit training on new data...")
print("-" * 60)

for epoch in range(3):  # Fewer epochs for partial fit
    # Simulate metrics that improve slowly (model already trained)
    train_loss = 0.3 + 0.1 * np.exp(-epoch) + np.random.normal(0, 0.01)
    val_acc = 0.85 + 0.05 * (1 - np.exp(-epoch)) + np.random.normal(0, 0.01)
    
    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_acc={val_acc:.4f}")

## Part 4: Fine-Tuning (Transfer Learning)

Adapt a model trained on one dataset to a different but related dataset.

In [ ]:
# Load a different dataset for fine-tuning
target_dsid = 'ECG200'
X_target_train, y_target_train, X_target_test, y_target_test = tsai_rs.get_UCR_data(
    target_dsid, return_split=True
)

target_n_vars = X_target_train.shape[1]
target_seq_len = X_target_train.shape[2]
target_n_classes = len(np.unique(y_target_train))

print(f"Target dataset: {target_dsid}")
print(f"Shape: {X_target_train.shape}")
print(f"Classes: {target_n_classes}")

In [ ]:
# Standardize target data
X_target_train_std = tsai_rs.ts_standardize(X_target_train.astype(np.float32), by_sample=True)
X_target_test_std = tsai_rs.ts_standardize(X_target_test.astype(np.float32), by_sample=True)

In [ ]:
# Create configuration for target dataset
# Note: Architecture may need adjustment for different input dimensions
target_model_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=target_n_vars,
    seq_len=target_seq_len,
    n_classes=target_n_classes
)

print(f"Target model config: {target_model_config}")

### Fine-Tuning Strategy

In [ ]:
# Fine-tuning typically uses:
# 1. Lower learning rate
# 2. Gradual unfreezing (train head first, then full model)
# 3. Discriminative learning rates (different LR for different layers)

fine_tune_strategies = {
    'Phase 1 - Train head only': {
        'lr': 1e-3,
        'epochs': 3,
        'freeze_backbone': True
    },
    'Phase 2 - Train full model': {
        'lr': 1e-4,
        'epochs': 5,
        'freeze_backbone': False
    }
}

print("Fine-tuning strategy:")
for phase, params in fine_tune_strategies.items():
    print(f"\n{phase}:")
    for key, value in params.items():
        print(f"  {key}: {value}")

In [ ]:
# Simulate fine-tuning
print("\nFine-tuning simulation:")
print("=" * 60)

# Phase 1: Train head only
print("\nPhase 1: Training classification head...")
print("-" * 40)
for epoch in range(3):
    train_loss = 0.8 * np.exp(-epoch) + np.random.normal(0, 0.02)
    val_acc = 0.6 + 0.2 * (1 - np.exp(-epoch)) + np.random.normal(0, 0.02)
    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_acc={val_acc:.4f}")

# Phase 2: Train full model
print("\nPhase 2: Training full model...")
print("-" * 40)
base_acc = 0.8
for epoch in range(5):
    train_loss = 0.4 * np.exp(-epoch/2) + np.random.normal(0, 0.01)
    val_acc = base_acc + 0.1 * (1 - np.exp(-epoch/2)) + np.random.normal(0, 0.01)
    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_acc={val_acc:.4f}")

print("\nFine-tuning complete!")

## Best Practices

In [ ]:
best_practices = {
    'Inference': [
        'Save model config alongside weights',
        'Standardize input data the same way as training',
        'Use batch inference for efficiency',
        'Consider model quantization for deployment'
    ],
    'Partial Fit': [
        'Use lower learning rate (1/10 of original)',
        'Train for fewer epochs',
        'Mix old and new data to prevent forgetting',
        'Monitor performance on old data'
    ],
    'Fine-Tuning': [
        'Start by training only the head',
        'Gradually unfreeze layers',
        'Use discriminative learning rates',
        'Early stopping to prevent overfitting'
    ]
}

print("Best Practices")
print("=" * 60)
for category, practices in best_practices.items():
    print(f"\n{category}:")
    for i, practice in enumerate(practices, 1):
        print(f"  {i}. {practice}")

## Summary

This notebook covered:

### Inference
```python
# Load configuration
config = tsai_rs.InceptionTimePlusConfig(
    n_vars=loaded_config['n_vars'],
    seq_len=loaded_config['seq_len'],
    n_classes=loaded_config['n_classes']
)

# Standardize input
X_std = tsai_rs.ts_standardize(X.astype(np.float32), by_sample=True)

# Generate predictions
probs, preds = model.predict(X_std)
```

### Partial Fit
```python
# Lower learning rate for incremental learning
config = tsai_rs.LearnerConfig(lr=1e-4, weight_decay=0.01)

# Train for fewer epochs on new data
```

### Fine-Tuning
```python
# Two-phase fine-tuning:
# 1. Train head with higher LR
# 2. Train full model with lower LR
```

In [ ]:
# Quick reference
print("Inference & Fine-Tuning Quick Reference")
print("=" * 50)
print("\n# Inference")
print("X_std = tsai_rs.ts_standardize(X.astype(np.float32), by_sample=True)")
print("\n# Partial fit config")
print("config = tsai_rs.LearnerConfig(lr=1e-4)  # Lower LR")
print("\n# Fine-tuning phases")
print("# Phase 1: Train head (lr=1e-3, freeze_backbone=True)")
print("# Phase 2: Full model (lr=1e-4, freeze_backbone=False)")

## Cleanup

In [ ]:
# Clean up saved files
import shutil

if models_path.exists():
    shutil.rmtree(models_path)
    print(f"Cleaned up {models_path}")